In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [2]:
from indox import IndoxRetrievalAugmentation

indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            


In [3]:
from indox.llms import OpenAi
from indox.embeddings import OpenAiEmbedding

openai_qa = OpenAi(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")
embed_openai = OpenAiEmbedding(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

INFO: Initializing OpenAi with model: gpt-3.5-turbo-0125
INFO: OpenAi initialized successfully
INFO: Initialized OpenAiEmbedding with model: text-embedding-3-small


In [ ]:
!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

In [4]:
file_path = "sample.txt"

In [5]:
from indox.data_loader_splitter import UnstructuredLoadAndSplit

loader_splitter = UnstructuredLoadAndSplit(file_path=file_path, max_chunk_size=400)
docs = loader_splitter.load_and_chunk()

INFO: UnstructuredLoadAndSplit initialized successfully
INFO: Getting all documents
INFO: Starting processing
INFO: Using title-based chunking
INFO: Completed chunking process
INFO: Successfully obtained all documents


Run Weaviate in docker
```bash
docker pull semitechnologies/weaviate:latest
```
```bash
docker run -d -p 8080:8080 semitechnologies/weaviate:latest
```
or if you have error with permission
```bash
docker run -d -p 8080:8080 -e AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED=true -e PERSISTENCE_DATA_PATH=/var/lib/weaviate -v /path/on/host:/var/lib/weaviate semitechnologies/weaviate:latest
```

In [16]:
import weaviate

client = weaviate.Client("http://localhost:8080")

In [17]:
from indox.vector_stores import Weaviate
db = Weaviate(embedding=embed_openai, index_name="Sample", text_key="test", client=client)

In [18]:
db.add(docs=docs)

INFO: Connection to the vector store database established successfully
INFO: Storing documents in the vector store
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using engine: text-embedding-3-small
INFO: Document added successfully to the vector store.
INFO: Documents stored successfully


In [19]:
query = "How Cinderella reach her happy ending?"
retriever = indox.QuestionAnswer(vector_database=db, llm=openai_qa, top_k=5)

In [20]:
answer = retriever.invoke(query)
context = retriever.context

INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using engine: text-embedding-3-small
INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Generating response
INFO: Response generated successfully
INFO: Query answered successfully


In [21]:
answer

"Cinderella reached her happy ending by being kind, patient, and having a pure heart. Despite facing mistreatment from her step-family, she remained humble and continued to do good deeds. With the help of a little white bird, she was able to receive what she wished for and planted a branch from a hazel bush on her mother's grave, which grew into a beautiful tree. Cinderella's pure heart and unwavering faith eventually led to her being chosen by the prince as his bride during a three-day festival where all the beautiful young girls in the country were invited. The two white doves also played a crucial role in identifying Cinderella as the true bride, leading to her happy ending of marrying the prince and living happily ever after."